In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import glob
path_dataset_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-1/train_png/*.png")
path_dataset_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-2/train_png/*.png")
path_dataset_3=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-3/train_png/*.png")
path_dataset_4_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-4-1/train_png/*.png")

path_dataset_4_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-4-2/train_png/*.png")
path_dataset_5_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-5-1-index-yok/train_png/*.png")

path_dataset_5_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-5-2/train_png/*.png")

In [3]:
train_png_paths=path_dataset_1+path_dataset_2+path_dataset_3+path_dataset_4_1+path_dataset_4_2+path_dataset_5_1+path_dataset_5_2
len(train_png_paths)

518

In [4]:
train_png_paths[0]

'C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-1/train_png\\026c97_0.png'

In [5]:
train_png_paths[0].split('\\')[-1][:-4]

'026c97_0'

In [6]:
import numpy as np
DATASET_FOLDER = "C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/"


path_csv = os.path.join(DATASET_FOLDER, "train.csv")
df_train = pd.read_csv(path_csv)
list_df=[]
for i in df_train.index:
    for ii in range(len(train_png_paths)):
        if df_train['image_id'][i]==train_png_paths[ii].split('\\')[-1][:-4]:
            list_df.append([train_png_paths[ii],df_train['label'][i]])

In [7]:
df=pd.DataFrame(list_df,columns=['image_path','label'])
df

,image_path,label
0,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
1,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
2,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
3,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
4,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,LAA
...,...,...
513,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,LAA
514,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
515,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
516,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE


In [8]:
df_copy=df.copy()
Fseries = pd.Series(df_copy.image_path, name="filepaths")

Lseries = pd.Series(df_copy.label, name="labels")
inme_data = pd.concat([Fseries,Lseries], axis=1)
inmer_df = pd.DataFrame(inme_data)
print(inmer_df.head())
print(inmer_df["labels"].value_counts())

#shape of datatset
inmer_df.shape

                                           filepaths labels
0  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
1  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
2  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
3  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
4  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...    LAA
CE     392
LAA    126
Name: labels, dtype: int64


(518, 2)

In [9]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
#tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from tensorflow.keras.callbacks import TensorBoard,EarlyStopping 

In [10]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(inmer_df, test_size=0.2, shuffle=True)


In [11]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_df,x_col="filepaths",y_col="labels",
                                      target_size=(128,128),

                                      class_mode='categorical', #used for Sequential Model
                                      batch_size=1,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_df,x_col="filepaths",y_col="labels",
                                      target_size=(128,128),

                                      class_mode='categorical', #used for Sequential Model
                                      batch_size=1,
                                      shuffle=False            #do not shuffle data
                                     )

Found 414 validated image filenames belonging to 2 classes.
Found 104 validated image filenames belonging to 2 classes.


In [12]:
## from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')
for layer in base_model.layers:
    layer.trainable = False
from tensorflow.keras.optimizers import RMSprop

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(2, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

87910968/87910968 [==============================] - 63s 1us/step


In [13]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=30
                             )
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="./class_model_EfficientNetB7.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )

In [14]:
inc_history = model.fit_generator(train, epochs = 10,validation_data=test,
                                        callbacks = [checkpointer, earlystopping, reduce_lr])

Epoch 1/10
414/414 [==============================] - ETA: 0s - loss: 1.5180 - acc: 0.6884
Epoch 1: val_loss improved from inf to 1.16223, saving model to .\class_model_EfficientNetB7.h5
414/414 [==============================] - 2884s 7s/step - loss: 1.5180 - acc: 0.6884 - val_loss: 1.1622 - val_acc: 0.7115 - lr: 1.0000e-04
Epoch 2/10
414/414 [==============================] - ETA: 0s - loss: 1.2619 - acc: 0.7222
Epoch 2: val_loss did not improve from 1.16223
414/414 [==============================] - 2855s 7s/step - loss: 1.2619 - acc: 0.7222 - val_loss: 1.1876 - val_acc: 0.6154 - lr: 1.0000e-04
Epoch 3/10
414/414 [==============================] - ETA: 0s - loss: 1.2930 - acc: 0.7464
Epoch 3: val_loss did not improve from 1.16223
414/414 [==============================] - 2820s 7s/step - loss: 1.2930 - acc: 0.7464 - val_loss: 1.9815 - val_acc: 0.7212 - lr: 1.0000e-04
Epoch 4/10
414/414 [==============================] - ETA: 0s - loss: 1.0805 - acc: 0.7802
Epoch 4: val_loss did not 

In [15]:
model.save('inceptionV3.h5')